# 要約 
このJupyter Notebookは、LMSYSのChatbot Arenaコンペティションにおける人間の好み予測のための機械学習モデルのトレーニングに取り組んでいます。具体的には、Llama-3言語モデルを基にしたファインチューニングを行い、与えられたプロンプトに対してユーザーが好む応答のモデルを構築しています。

### 問題の背景
本コンペティションでは、ユーザーが提示したプロンプトに対する異なる応答の中から、どちらの応答が好まれるかを予測することが求められます。この予測は、ログ損失を用いて評価されます。

### 手法
このNotebookでは以下の手法とライブラリを使用しています：

- **ライブラリ**:
  - `transformers`: 大規模言語モデルやトークナイザーを用いるために使用。
  - `datasets`: データの読み込みと前処理を行うために利用。
  - `scikit-learn`: ログ損失や精度などのメトリクス計算を行うために使用。
  - `torch`: PyTorchによるニューラルネットワークの構築および訓練に使用。

- **モデル**:
  - Llama-3ベースモデル（`llama-3-8b-Instruct-bnb-4bit`）を利用し、これをファインチューニングして人間の選好を予測するモデルを構築しています。

- **データ処理**:
  - データセットからプロンプトと応答を抽出し、それらをトークン化。
  - 各応答に対するラベル（「winner_model_a」「winner_model_b」「winner_tie」）を生成。

- **トレーニング戦略**:
  - 交差検証を用いてモデルの性能を評価。
  - 勾配蓄積やバッチサイズ、エポック数を調整し、効率的にトレーニング。

- **評価**:
  - 訓練後、ログ損失と精度を検証し、最終的なパフォーマンスを評価しています。

### 結果
Notebookの結果セクションでは、推論コードへのリンクと評価データセットおよびリーダーボードでのログ損失（Eval: 0.9231, LB: 0.936）が示されています。また、成功裏に再現可能な設定として、使用するデータやハイパーパラメータ（例：トレーニング中のバッチサイズ、エポック数など）が明記されています。

このNotebookは、提供された情報に基づいて、ユーザーに好まれるチャットボットの応答を予測するための効率的な機械学習アプローチを示しており、実用的な応用が期待されます。

---


# 用語概説 
以下は、Jupyter Notebookの内容に基づいて、機械学習・深層学習初心者がつまずきそうな専門用語の簡単な解説です。

1. **Log Loss**:
   - **解説**: ログ損失は、分類問題の評価指標の一つ。予測確率と実際のラベルとの間の対数損失を計算します。小さいほど良いモデルを示します。

2. **Label Encoder**:
   - **解説**: ラベルエンコーダは、カテゴリ変数を数値形式に変換するためのツール。例えば、文字列のクラスラベルを整数に変換し、機械学習モデルに入力できるようにします。

3. **Causal Language Model (CAUSAL_LM)**:
   - **解説**: 原因のある言語モデルで、与えられた文脈に基づいて次に来る単語を予測するモデル。例えば、文中の前の単語を考慮しながら次の単語を生成します。

4. **Lora**:
   - **解説**: Lora（Low-Rank Adaptation）は大規模なモデルのトレーニングを効率化する手法。モデルの重みを低ランクのパラメータで補正し、計算コストを削減します。

5. **k-bit Training**:
   - **解説**: k-bitトレーニングは、モデルの重みをkビットで表現する手法。これにより、メモリ消費量を削減し、高速でトレーニングが行えるようになります。

6. **Attention Mask**:
   - **解説**: 注意マスクは、トークン間での注意計算において、どのトークンを無視するべきかを示すマスク。無視したいトークンをマスクすることで、モデルが効率的に学習できるようにします。

7. **Gradient Accumulation**:
   - **解説**: 勾配蓄積は、通常のバッチサイズよりも小さいバッチでトレーニングを行った後、複数の勾配を蓄積して一度に更新する手法。これにより、大規模データセットでのメモリ効率を改善できます。

8. **Data Collator**:
   - **解説**: データコレータは、ミニバッチを作成するためのツール。異なるサイズの入力を均一なバッチに整形し、モデルに渡す役割を担います。

9. **EvalPrediction**:
   - **解説**: EvalPredictionは、評価プロセス中に使われるデータ構造で、モデルの予測と関連ラベルを含む。これにより評価メトリクスを計算できます。

10. **Softmax Function**:
    - **解説**: ソフトマックス関数は、モデルの出力を確率に変換するための関数。出力の和が1になるように正規化し、クラス分類タスクにおいて有用です。

これらの用語は、Jupyter Notebookの特定の操作や機能に関連しており、初心者が理解しにくい部分です。十分な前知識を持つ読者にとっても重要な概念となるでしょう。

---


## 結果
- [推論コード](https://www.kaggle.com/code/shelterw/sft-llama-3-8b-inference)    

- [ベースモデル: llama-3-8b-Instruct-bnb-4bit](https://huggingface.co/unsloth/llama-3-8b-Instruct-bnb-4bit)

| サブセット | log loss |
| - | - |
| Eval | 0.9231 |
| LB | 0.936 |

## 注意
コードを再現したい場合は、以下の点に注意してください：
- すべてのデータを使用する
- per_device_train_batch_size=4を設定する
- 1エポックのトレーニングにはA10を使用して約15時間かかりました



In [ ]:
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [ ]:
import os
import copy
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from datasets import Dataset
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BitsAndBytesConfig,
    LlamaPreTrainedModel,
    LlamaModel,
    AutoTokenizer,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
from transformers.modeling_outputs import CausalLMOutputWithPast
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

### 設定


In [ ]:
TRAIN_CSV = "/kaggle/input/lmsys-chatbot-arena/train.csv"
model_path = "unsloth/llama-3-8b-Instruct-bnb-4bit"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 1024
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']
columns_to_vectorize = ["prompt", "response_a", "response_b"]

train = pd.read_csv(TRAIN_CSV)
train = train.head(100)
train['label'] = train[target_columns].idxmax(axis=1) 
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
train = train[columns_to_vectorize + ['label']]

### トークナイザーとデータセットの準備、メトリクス


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.add_eos_token = True
tokenizer.padding_side = 'right'

# ラベルに対応するトークンIDを取得
LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]

# トークン化の関数を定義
def tokenize(example, tokenizer):
    # プロンプトと応答をトークン化
    prompt = tokenizer('<prompt>: ' + " ".join(eval(example['prompt'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_a = tokenizer('\n\n<response_a>: ' + " ".join(eval(example['response_a'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_b = tokenizer('\n\n<response_b>: ' + " ".join(eval(example['response_b'], {"null": ""})), add_special_tokens=False)["input_ids"]
    
    # トークンIDの長さが最大長を超える場合、トリミング
    if len(prompt+response_a+response_b) > MAX_LENGTH:
        prompt = tokenizer('<prompt>: ' + eval(example['prompt'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:256]
        response_a = tokenizer('\n\n<response_a>: ' + eval(example['response_a'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
        response_b = tokenizer('\n\n<response_b>: ' + eval(example['response_b'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
        
    # 追加のプロンプトを定義
    extra_prompt = tokenizer('\n\n---------\nWhich is the better response for the prompt ? a or b or tie ?\n\nAnswer: ', add_special_tokens=False)["input_ids"]

    # ラベルのトークンIDを取得
    label_token_id = LABEL_IDS[int(example['label'])]
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + [label_token_id] + [tokenizer.eos_token_id]
    attention_mask = len(input_ids)*[1]
    labels = [-100]* len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + [label_token_id] + [tokenizer.eos_token_id]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
# データを読み込み、トークン化する関数を定義
def load_data(df, tokenizer):
    raw_datasets = Dataset.from_pandas(df)  # pandas DataFrameをDatasetに変換
    tokenized_datasets = raw_datasets.map(
        tokenize,  # トークナイズ関数を適用
        remove_columns=raw_datasets.column_names,
        fn_kwargs={'tokenizer': tokenizer}
    )
    return tokenized_datasets

# メトリクスを計算する関数を定義
def compute_metrics(pred):
    logits, labels = pred  # 予測とラベルを取得
    preds = logits.argmax(axis=-1)  # 最も高いロジットを持つインデックスを予測
    label_tokens_ids = np.array(LABEL_IDS)
    index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
    labels = labels[np.isin(labels, label_tokens_ids)]
    labels = np.array([index_mapping[label.item()] for label in labels])  # ラベルをインデックスにマッピング
    
    acc = accuracy_score(labels, preds)  # 精度を計算
    probs = softmax(logits, axis=-1)  # ソフトマックスを計算
    log_loss_ = log_loss(labels, probs)  # log lossを計算
    return {'accuracy': acc, 'log_loss': log_loss_}

n_splits = 5
fold_idx = 0
ds = load_data(train, tokenizer)  # データの読み込み
folds = [
    (
        [i for i in range(len(ds)) if i % n_splits != fold_idx],
        [i for i in range(len(ds)) if i % n_splits == fold_idx]
    ) 
    for fold_idx in range(n_splits)  # n分割交差検証のインデックスを生成
]
train_idx, eval_idx = folds[fold_idx]  # トレーニングと評価のインデックスを分ける

### モデル


In [ ]:
class Llama3ForSFT(LlamaPreTrainedModel):
    _tied_weights_keys = ["lm_head.weight"]  # 重みを共有するためのキー
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)  # Llamaモデルの初期化
        self.vocab_size = config.vocab_size  # 語彙サイズの取得
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)  # 線形層の初期化
        self.post_init()

    def forward(
        self,
        input_ids= None,
        attention_mask= None,
        position_ids = None,
        past_key_values= None,
        inputs_embeds= None,
        labels= None,
        use_cache= None,
        output_attentions= None,
        output_hidden_states = None,
        return_dict= None,
        cache_position = None,
    ):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        
        hidden_states = outputs[0]  # 隠れ状態を取得
        if self.config.pretraining_tp > 1:
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
            logits = torch.cat(logits, dim=-1)
        else:
            logits = self.lm_head(hidden_states)  # 隠れ状態からロジットを取得
        logits = logits.float()  # ロジットを浮動小数点に変換

        loss = None
        if labels is not None:
            # トークンをずらして予測に使用
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # トークンをフラット化
            loss_fct = nn.CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # モデルの並列処理を有効にする
            shift_labels = shift_labels.to(shift_logits.device)

            label_tokens_ids = torch.tensor(LABEL_IDS, device=shift_labels.device)
            index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
            true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]
            true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)
            true_logits = shift_logits[torch.isin(shift_labels, label_tokens_ids)][:, label_tokens_ids]
            loss = loss_fct(true_logits, true_labels)  # ロスを計算

        return CausalLMOutputWithPast(
            loss=loss,
            logits=true_logits,
        )

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj', 'k_proj', 'v_proj',], 
)

In [ ]:
model = Llama3ForSFT.from_pretrained(
    model_path, 
    load_in_8bit=True,
    torch_dtype=torch.float16,
    cache_dir="/kaggle/working/model"
)
model.config.use_cache = False  # キャッシュを使用しないよう設定
model = prepare_model_for_kbit_training(model)  # k-bitトレーニングのためにモデルを準備
model = get_peft_model(model, peft_config)  # Loraモデルを取得
print(model)  # モデルの構造を表示
model.print_trainable_parameters()  # 訓練可能なパラメータを表示

#### 学習引数


In [ ]:
args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir = True,  # 出力ディレクトリを上書き
    evaluation_strategy = "epoch",  # 評価戦略
    save_strategy = "steps",  # 保存戦略
    save_steps=200,  # 保存するステップ数
    save_total_limit=1,  # 保存するモデルの最大数
    logging_strategy="steps",  # ロギング戦略
    logging_steps=10,  # ロギングするステップ数
    warmup_steps=20,  # ウォームアップステップ数
    optim="adamw_8bit",  # オプティマイザ
    learning_rate=2e-4,  # 学習率
    per_device_train_batch_size=2,  # デバイスごとの訓練バッチサイズ
    per_device_eval_batch_size=4,  # デバイスごとの評価バッチサイズ
    gradient_accumulation_steps=2,  # 勾配蓄積ステップ数
    num_train_epochs=1,  # 訓練エポック数
    fp16=True,  # 16ビット浮動小数点の使用
    metric_for_best_model="log_loss",  # ベストモデルのメトリック
    greater_is_better = False,  # メトリックが大きい方が良いかどうか
    report_to="none",
)

### 学習中 !


In [ ]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=ds.select(train_idx),  # トレーニングデータセット
    eval_dataset=ds.select(eval_idx),  # 評価データセット
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # データコレータ
    compute_metrics=compute_metrics,  # メトリクス計算の関数
)
trainer.train()  # 学習を開始

In [ ]:
model.save_pretrained('pretrained_model')  # 学習したモデルを保存

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import LoraConfig, get_peft_model, TaskType

# # 事前に学習されたモデルとLoraアダプタのパス
# model_path = "unsloth/llama-3-8b-Instruct-bnb-4bit"
# lora_adapter_path = "/kaggle/input/model-1"

# # ベースモデルをロード
# model_1 = AutoModelForCausalLM.from_pretrained(model_path)

# # トークナイザーをロード
# tokenizer = AutoTokenizer.from_pretrained(model_path)

# # Loraの設定
# lora_config = LoraConfig(
#     r=8,            # Loraのランク
#     lora_alpha=16,  # Loraのスケーリングファクター
#     task_type=TaskType.CAUSAL_LM  # モデルのタスクタイプ
# )

# # モデルをk-bitトレーニング用に準備
# model_1 = prepare_model_for_kbit_training(model_1)

# # モデルにLoraアダプタを適用
# model_1 = get_peft_model(model_1, lora_config)

# # 保存されたLoraアダプタのパラメータをロード
# model_1.load_adapter(lora_adapter_path, adapter_name="test")

# # モデルが使用可能になった
# model_1.eval()  # 評価モードに設定

# # 例文をトークン化してテキストを生成
# sentence = "Hello, how are you?"
# inputs = tokenizer(sentence, return_tensors="pt")
# outputs = model_1.generate(**inputs)

# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# !zip -r model_2.zip /kaggle/working/saved_model_2